# Выбор локации для скважины

Для добывающей сырье компании нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Требуется построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Нужно также проанализировать возможные прибыль и риски.

**Шаги для выбора локации:**

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

**Условия задачи:**
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

**Значения столбцов:**
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## 1. Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np 

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from numpy.random import RandomState

pd.options.mode.chained_assignment=None

In [2]:
try:
    df0 = pd.read_csv('/datasets/geo_data_0.csv') # читаем csv-файл и сохраняем в переменную df
except:
    df0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv') 

df0.sample(5) # смотрим случайные 5 строк из датасета для ознакомления

,id,f0,f1,f2,product
94400,broNg,-0.409803,0.853466,0.475550,47.032384
60980,ReLhH,-0.339466,0.901200,7.289915,111.377611
20272,OMfbg,0.490629,-0.391895,-1.050727,106.195451
58523,eLZiV,-0.825188,0.264688,3.038618,61.147705
24177,0TXkv,0.028340,0.103588,4.505864,103.184892


In [3]:
df0.info() # пропусков нет, названия столбцов в порядке, форматы данных адекватные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [4]:
df0.describe() # совсем откровенных аномалий (вроде отрицательного значения product) не наблюдается

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


In [5]:
df0.corr() # даже максимальные коэффициенты корреляции между разными признаками ниже значения в 0.5 по модулю, для   
# подозрений же о наличии мультиколлинеарности нужен уровень выше хотя бы 0.7; конечно только расчета коэффициентов корреляции не 
# достаточно, чтобы полностью отбросить риски наличия мультиколлинеарности, но в рамках наших текущих знаний - это все, 
# что мы можем проанализировать

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [6]:
df0.duplicated().sum() # проверяем наличие полных дубликатов

0

In [7]:
df0.id.nunique() # посмотрим нет ли дубликатов в столбце id

99990

In [8]:
duplicated_id_region0 = df0.groupby('id')['id'].value_counts().sort_values(ascending=False).head(10).unstack()
df0_duplicated_id = df0[df0['id'].isin(duplicated_id_region0.index)].sort_values(by='id')
df0_duplicated_id

,id,f0,f1,f2,product
66136,74z30,1.084962,-0.312358,6.990771,127.643327
64022,74z30,0.741456,0.459229,5.153109,140.771492
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
931,HZww2,0.755284,0.368511,1.863211,30.681774
7530,HZww2,1.061194,-0.373969,10.430210,158.828695
63593,QcMuo,0.635635,-0.473422,0.862670,64.578675
1949,QcMuo,0.506563,-0.323775,-2.215583,75.496502


In [9]:
# удаляем объекты с задублированными id
df0 = df0.drop(df0_duplicated_id.index)
df0.shape

(99980, 5)

In [10]:
df0.id.nunique() # проверяем отсутствие дубликатов

99980

In [11]:
try:
    df1 = pd.read_csv('/datasets/geo_data_1.csv') # читаем csv-файл и сохраняем в переменную df
except:
    df1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv') 

df1.sample(5) # смотрим случайные 5 строк из датасета для ознакомления

,id,f0,f1,f2,product
62377,oa1As,13.327970,-1.301379,0.008769,0.000000
83410,m5rTF,3.177526,-1.795695,-0.004163,0.000000
97011,bp2Zk,-6.251172,-12.491243,1.992214,57.085625
72491,LWx9j,-5.423087,-4.090863,1.996071,57.085625
78610,q1ySJ,-2.390464,-1.749470,3.999152,110.992147


In [12]:
df1.info() # пропусков нет, названия столбцов в порядке, форматы данных адекватные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [13]:
df1.describe() # совсем откровенных аномалий (вроде отрицательного значения product) не наблюдается

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


In [14]:
df1.corr() # даже максимальные коэффициенты корреляции между разными признаками ниже значения в 0.2 по модулю, для   
# подозрений же о наличии мультиколлинеарности нужен уровень выше хотя бы 0.7; конечно только расчета коэффициентов корреляции не 
# достаточно, чтобы полностью отбросить риски наличия мультиколлинеарности, но в рамках наших текущих знаний - это все, 
# что мы можем проанализировать

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [15]:
df1.duplicated().sum() # проверяем наличие полных дубликатов

0

In [16]:
df1.id.nunique() # посмотрим нет ли дубликатов в столбце id

99996

In [17]:
duplicated_id_region1 = df1.groupby('id')['id'].value_counts().sort_values(ascending=False).head(4).unstack()
df1_duplicated_id = df1[df1['id'].isin(duplicated_id_region1.index)].sort_values(by='id')
df1_duplicated_id

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [18]:
# удаляем объекты с задублированными id
df1 = df1.drop(df1_duplicated_id.index)
df1.shape

(99992, 5)

In [19]:
df1.id.nunique() # проверяем отсутствие дубликатов

99992

In [20]:
try:
    df2 = pd.read_csv('/datasets/geo_data_2.csv') # читаем csv-файл и сохраняем в переменную df
except:
    df2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv') 

df2.sample(5) # смотрим случайные 5 строк из датасета для ознакомления

,id,f0,f1,f2,product
5631,hHJWx,-3.965570,0.561443,0.805358,105.210251
57700,lKEyU,-0.403480,-2.054513,3.102665,44.421004
12301,PWob2,1.748170,-1.943102,-0.350699,91.156240
31462,FGdga,-1.210856,1.303434,-0.478658,34.724857
83946,DFOkk,-1.486918,0.113909,2.465207,42.192526


In [21]:
df2.info() # пропусков нет, названия столбцов в порядке, форматы данных адекватные

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [22]:
df2.describe() # совсем откровенных аномалий (вроде отрицательного значения product) не наблюдается

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [23]:
df2.corr() # коэффициенты корреляции между разными признаками близки к 0, для подозрений же о наличии мультиколлинеарности 
# нужен уровень выше хотя бы 0.7; конечно только расчета коэффициентов корреляции не достаточно, чтобы полностью отбросить риски 
# наличия мультиколлинеарности, но в рамках наших текущих знаний - это все, что мы можем проанализировать

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


In [24]:
df2.duplicated().sum() # проверяем наличие полных дубликатов

0

In [25]:
df2.id.nunique() # посмотрим нет ли дубликатов в столбце id

99996

In [26]:
duplicated_id_region2 = df2.groupby('id')['id'].value_counts().sort_values(ascending=False).head(4).unstack()
df2_duplicated_id = df2[df2['id'].isin(duplicated_id_region2.index)].sort_values(by='id')
df1_duplicated_id

,id,f0,f1,f2,product
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [27]:
# удаляем объекты с задублированными id
df2 = df2.drop(df2_duplicated_id.index)
df2.shape

(99992, 5)

In [28]:
df1.id.nunique() # проверяем отсутствие дубликатов

99992

**Вывод:** Значимых уровней линейной зависимости между фичами обнаружено не было, так что считаем, что проблема мультиколлинеарности не стоит. Полных дубликатов не выявлено, однако обнаружены строки с одинаковыми ID, так как количество их незначительно и невозможно определить, какая из срок с одинаковыми ID является корректной, пришлось удалить каждую пару одинаковых ID.

**Вывод:** Пропусков нет, названия столбцов в порядке, форматы данных адекватные, откровенных аномалий не наблюдается - двигаемся дальше.

## 2. Обучение и проверка модели для каждого региона

 ### 2.1. Разбивка данных на обучающую и валидационную выборки в соотношении 75:25.

In [29]:
ts = 0.25 # заносим значение test_size в переменную ts
rs = 12345 # заносим значение random_state в переменную rs

In [30]:
# использовал функцию для однотипнх вычислений после комментария ревьюера

def train_valid_spit(df):    
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target,
    test_size=ts, random_state=rs)
    return features_train, features_valid, target_train, target_valid

features_train0, features_valid0, target_train0, target_valid0 = train_valid_spit(df0)
features_train1, features_valid1, target_train1, target_valid1 = train_valid_spit(df1)
features_train2, features_valid2, target_train2, target_valid2 = train_valid_spit(df2)

In [31]:
# использовал функцию для однотипнх вычислений после комментария ревьюера

def scale(features_train, features_valid):
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    return features_train, features_valid

features_train0, features_valid0 = scale(features_train0, features_valid0)
features_train1, features_valid1 = scale(features_train1, features_valid1)
features_train2, features_valid2 = scale(features_train2, features_valid2)

 ### 2.2. Обучение модели и получение предсказаний на валидационной выборке.

In [32]:
# использовал функцию для однотипнх вычислений после комментария ревьюера

def pred_val(features_train, target_train, features_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    return predictions_valid

predictions_valid0 = pred_val(features_train0, target_train0, features_valid0)
predictions_valid1 = pred_val(features_train1, target_train1, features_valid1)
predictions_valid2 = pred_val(features_train2, target_train2, features_valid2)

 ### 2.3. Сохранение предсказаний и правильных ответов на валидационной выборке.

Предсказания и правильные ответы на валидационной выборке уже сохранены в соответствующих переменных на предыдущих шагах.

 ### 2.4. Вывод на экран среднего запаса предсказанного сырья и RMSE модели.

In [33]:
# добавил средний запас фактического сырья после комментария ревьюера:
meantarget0 = target_valid0.mean()
# считаем средний запас предсказанного сырья: 
meanpred0 = predictions_valid0.mean()
# посчитаем значение метрики RMSE на валидационной выборке:
rmse0 = mean_squared_error(target_valid0, predictions_valid0) ** 0.5
print('Средний запас предсказанного сырья для региона 0:', round(meanpred0, 2), 'тыс. баррелей')
print('Средний запас фактического сырья для региона 0:', round(meantarget0, 2), 'тыс. баррелей')
print('RMSE модели линейной регрессии для региона 0 на валидационной выборке:', round(rmse0, 2), 'тыс. баррелей')

Средний запас предсказанного сырья для региона 0: 92.42 тыс. баррелей
Средний запас фактического сырья для региона 0: 92.39 тыс. баррелей
RMSE модели линейной регрессии для региона 0 на валидационной выборке: 37.72 тыс. баррелей


In [34]:
# добавил средний запас фактического сырья после комментария ревьюера:
meantarget1 = target_valid1.mean()
# считаем средний запас предсказанного сырья:
meanpred1 = predictions_valid1.mean()
# посчитаем значение метрики RMSE на валидационной выборке:
rmse1 = mean_squared_error(target_valid1, predictions_valid1) ** 0.5
print('Средний запас предсказанного сырья для региона 1:', round(meanpred1, 2), 'тыс. баррелей')
print('Средний запас фактического сырья для региона 0:', round(meantarget1, 2), 'тыс. баррелей')
print('RMSE модели линейной регрессии для региона 1 на валидационной выборке:', round(rmse1, 2), 'тыс. баррелей')

Средний запас предсказанного сырья для региона 1: 68.98 тыс. баррелей
Средний запас фактического сырья для региона 0: 68.98 тыс. баррелей
RMSE модели линейной регрессии для региона 1 на валидационной выборке: 0.89 тыс. баррелей


In [35]:
# добавил средний запас фактического сырья после комментария ревьюера:
meantarget2 = target_valid2.mean()
# считаем средний запас предсказанного сырья:
meanpred2 = predictions_valid2.mean()
# посчитаем значение метрики RMSE на валидационной выборке:
rmse2 = mean_squared_error(target_valid2, predictions_valid2) ** 0.5
print('Средний запас предсказанного сырья для региона 2:', round(meanpred2, 2), 'тыс. баррелей')
print('Средний запас фактического сырья для региона 0:', round(meantarget2, 2), 'тыс. баррелей')
print('RMSE модели линейной регрессии для региона 2 на валидационной выборке:', round(rmse2, 2), 'тыс. баррелей')

Средний запас предсказанного сырья для региона 2: 95.12 тыс. баррелей
Средний запас фактического сырья для региона 0: 94.55 тыс. баррелей
RMSE модели линейной регрессии для региона 2 на валидационной выборке: 39.98 тыс. баррелей


**Сравнение предсказанных средних запасов по регионам с реальными:** Значения средних из почти 25000 предсказанных запасов сырья для каждого региона оказались поразительно близки к значениям реальных средних запасов из валидационной выборки для тех же регионов. Скорее всего это связано с достаточно большим объемом данных (одно из проявлений закона больших чисел): отдельные предсказания могут отклоняться достаточно значительно, судя по значениям метрики RMSE, особенно для регионов 0 и 2 (отклонения могут составить до примерно 40% от среднего значения), однако при расчете среднего отклонения могут компенсировать друг друга и его значение получается близким к фактическому среднему.  

 ### 2.5. Анализ результатов.

Лучший регион по средним фактическим запасам - регион2 (94.55 тыс. баррелей), немного отстает регион0 (92.39 тыс. баррелей) и с ощутимым отставанием идет регион1 (68.98 тыс. баррелей). Однако по показателю RMSE как раз регион1 лидирует с сильным перевесом (0.89 тыс. баррелей), тогда как регион0 и регион2 идут почти вровень (37.72 и 39.98 тыс. баррелей соответственно).

## 3. Подготовка к расчёту прибыли

### 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [36]:
points_to_research = 500 # при разведке региона исследуют 500 точек
points_to_develop = 200 # с помощью машинного обучения выбирают 200 лучших для разработки
development_budget = 10000000000 # бюджет на разработку скважин в регионе — 10 млрд рублей
revenue_per_product_unit = 450000 # доход с каждой единицы продукта составляет 450 тыс. руб.
loss_risk_threshold = 0.025 # нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%

### 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [37]:
# считаем порог безубыточности для региона и для каждой скважины в среднем:
regional_breakeven_treshold = development_budget / revenue_per_product_unit
point_breakeven_treshold = regional_breakeven_treshold / points_to_develop

print('Достаточный объём сырья с региона для безубыточной разработки скважин:', round(regional_breakeven_treshold, 2), \
     'тыс. баррелей')
print('Достаточный объём сырья со скважины для безубыточной разработки новой скважины:', round(point_breakeven_treshold, 2), \
     'тыс. баррелей')    

Достаточный объём сырья с региона для безубыточной разработки скважин: 22222.22 тыс. баррелей
Достаточный объём сырья со скважины для безубыточной разработки новой скважины: 111.11 тыс. баррелей


In [38]:
# сравним полученный порог со средним запасом в регионе:
print('Средний фактический запас скважин в регионе 0:', round(meantarget0, 2), 'тыс. баррелей')
if meantarget0 >= point_breakeven_treshold:
    print('Является достаточным для безубыточной разработки.')
else:
    print('Является недостаточным для безубыточной разработки.')

Средний фактический запас скважин в регионе 0: 92.39 тыс. баррелей
Является недостаточным для безубыточной разработки.


In [39]:
# сравним полученный порог со средним запасом в регионе:
print('Средний фактический запас скважин в регионе 1:', round(meantarget1, 2), 'тыс. баррелей')
if meantarget1 >= point_breakeven_treshold:
    print('Является достаточным для безубыточной разработки.')
else:
    print('Является недостаточным для безубыточной разработки.')

Средний фактический запас скважин в регионе 1: 68.98 тыс. баррелей
Является недостаточным для безубыточной разработки.


In [40]:
# сравним полученный порог со средним запасом в регионе:
print('Средний фактический запас скважин в регионе 2:', round(meantarget2, 2), 'тыс. баррелей')
if meantarget2 >= point_breakeven_treshold:
    print('Является достаточным для безубыточной разработки.')
else:
    print('Является недостаточным для безубыточной разработки.')

Средний фактический запас скважин в регионе 2: 94.55 тыс. баррелей
Является недостаточным для безубыточной разработки.


**Вывод:** Порог для безубыточной разработки составляет 22 222.22 тыс. баррелей для региона (суммарно 200 скважин) и соответственно 111.11 тыс. баррелей в среднем для каждой скважины (из конечных 200, которые будут отобраны). Средние предсказанные запасы по всем скважинам во всех 3х регионах ниже значения порога для безубыточной разработки. Однако при расчете среднего из 25000 скважин (размер валидационной выборки) на него может влиять большое кол-во скважин с низким потенциалом, нам же нужно выбрать для разработки 200 лучших по показателям.

## 4. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

### 4.1., 4.2. Выберем скважины с максимальными значениями предсказаний и просуммируем целевое значение объёма сырья, соответствующее этим предсказаниям.

In [41]:
# задаем функцию для расчета прибыли по выбранным скважинам, на вход она принимает соответствующие предсказания на валидационной в-ке
def revenue(predictions_valid, target_valid):
    predicted_best200 = pd.Series(predictions_valid).sort_values(ascending=False)[:points_to_develop]
    target_best200 = (target_valid.reset_index(drop = True)[predicted_best200.index])
    total_product = target_best200.sum()
    return total_product * revenue_per_product_unit - development_budget

In [42]:
pre = pd.DataFrame({'pred': [1, 1]}, index = [0, 0])
tar = pd.DataFrame({'target': [1, 1]}, index = [0, 0])

tar.loc[pre.index]

,target
0,1
0,1
0,1
0,1


### 4.3. Рассчитаем прибыль для полученного объёма сырья.

In [43]:
print('Суммарная фактическая прибыль с лучших 200 предсказанных скважин в регионе0:', \
      round(revenue(predictions_valid0, target_valid0), 2))
print('Суммарная фактическая прибыль с лучших 200 предсказанных скважин в регионе1:', \
      round(revenue(predictions_valid1, target_valid1), 2))
print('Суммарная пфактическая прибыль с лучших 200 предсказанных скважин в регионе2:', \
      round(revenue(predictions_valid2, target_valid2), 2))

Суммарная фактическая прибыль с лучших 200 предсказанных скважин в регионе0: 3136026056.66
Суммарная фактическая прибыль с лучших 200 предсказанных скважин в регионе1: 2415086696.68
Суммарная пфактическая прибыль с лучших 200 предсказанных скважин в регионе2: 2465945792.01


**Вывод:** Если выбирать 200 лучших по предсказанным запасам скважин из всех почти 25000 скважин из валидационной выборки по региону, то скважины по всем 3 регионам обещают положительную прибыль. Лучший показатель у региона0 (3.14 млрд. руб.), ощутимо отстают регион2 (2.47 млрд. руб) и регион1 (почти 2.42 млрд. руб.). Однако возможна разведка только 500 рандомных скважин в каждом регионе, посмотрим, какие показатели будут по 200 лучшим, выбранным из 500 случайных. 

## 5. Посчитаем риски и прибыль для каждого региона:

### 5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

In [44]:
state = np.random.RandomState(rs) # задаем значение случайности для сэмплирования подвыборок
# задаем функцию, чтобы избежать повторяющегося кода:    
def revenue_bootstrap(predictions_valid, target_valid):
    values = []                                                          # сэмплируем подвыборки по 500 скважин
    for i in range(1000):
        predictions_valid_subsample = pd.Series(predictions_valid) \
        .sample(n=points_to_research, replace=True, random_state=state)   # тут мне кажется корректнее использовать
        values.append(revenue(predictions_valid_subsample, target_valid)) # replace=False, но оставил, как было в теории
# собираем значения прибыли на всех 1000 подвыборках по 500 штук                                                                     
    values = pd.Series(values)
# получаем среднее значение
    mean = values.mean()
# считаем 0.025- и 0.975-квантиль для определения 95%-го доверительного интервала 
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
# считаем вероятность отрицательной прибыли на 1000 подвыборок по 500 скважин с помощью самописной функции
    loss_risk = values.apply(lambda x: x < 0).sum()/len(values)

    return mean, lower, upper, loss_risk

mean0, lower0, upper0, loss_risk0 = revenue_bootstrap(predictions_valid0, target_valid0)
mean1, lower1, upper1, loss_risk1 = revenue_bootstrap(predictions_valid1, target_valid1)
mean2, lower2, upper2, loss_risk2 = revenue_bootstrap(predictions_valid0, target_valid0)


### 5.2. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

In [45]:
print('Данные по региону0:')
print(' ')
print('Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок:', round(mean0, 2))
print('95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от', round(lower0, 2),'до', \
      round(upper0, 2))
print('Риск убытков по региону:', round(loss_risk0, 2))
if loss_risk0 >= loss_risk_threshold:
    print('Регион не проходит по уровню риска убытков.')
else:
    print('Регион проходит по уровню риска убытков.')
print(' ')
print('Данные по региону1:')
print(' ')
print('Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок:', round(mean1, 2))
print('95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от', round(lower1, 2),'до', \
      round(upper1, 2))
print('Риск убытков по региону:', round(loss_risk1, 2))
if loss_risk1 >= loss_risk_threshold:
    print('Регион не проходит по уровню риска убытков.')
else:
    print('Регион проходит по уровню риска убытков.')
print(' ')
print('Данные по региону2:')
print(' ')
print('Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок:', round(mean2, 2))
print('95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от', round(lower2, 2),'до', \
      round(upper2, 2))
print('Риск убытков по региону:', round(loss_risk2, 2))
if loss_risk2 >= loss_risk_threshold:
    print('Регион не проходит по уровню риска убытков.')
else:
    print('Регион проходит по уровню риска убытков.')
print(' ')

Данные по региону0:
 
Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок: 431538186.58
95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от -80924626.51 до 941037638.4
Риск убытков по региону: 0.06
Регион не проходит по уровню риска убытков.
 
Данные по региону1:
 
Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок: 465108803.27
95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от 80299334.72 до 864399761.38
Риск убытков по региону: 0.01
Регион проходит по уровню риска убытков.
 
Данные по региону2:
 
Средняя суммарная предсказанная прибыль с лучших 200 скважин в регионе из 1000 подвыборок: 426894654.75
95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин в регионе: от -114359139.73 до 936751736.17
Риск убытков по региону: 0.05
Регион не проходит по уровню риска убытков.
 


### 5.3. Напишем выводы: предложим регион для разработки скважин и обоснуем выбор.

In [46]:
print('Сопоставим полученные параметры по регионам (0|1|2):')
print(' ')
print('Средний запас фактического сырья по регионам из выборки 25000 скважин:')
print(round(meantarget0, 2), '|', round(meantarget1, 2), '|', round(meantarget2, 2))
print(' ')
print('RMSE предсказаний по регионам:') 
print(round(rmse0, 2), '|', round(rmse1, 2), '|', round(rmse2, 2))
print(' ')
print('Суммарная фактическая прибыль с лучших 200 предсказанных скважин по регионам из выборки 25000 скважин:')
print(round(revenue(predictions_valid0, target_valid0), 2), '|', round(revenue(predictions_valid1, target_valid1), 2), \
      '|', round(revenue(predictions_valid2, target_valid2), 2))
print(' ')
print('Средняя суммарная фактическая прибыль с 200 лучших скважин по региону из 1000 подвыборок по 500 скважин:')
print(round(mean0, 2), '|', round(mean1, 2), '|', round(mean2, 2))
print(' ')
print('95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин по региону:')
print(round(lower0, 2),'до', round(upper0, 2), '|', round(lower1, 2),'до', round(upper1, 2), '|', round(lower2, 2), \
      'до', round(upper2, 2))
print(' ')
print('Риск убытков по региону:')
print(round(loss_risk0, 2), '|', round(loss_risk1, 2), '|', round(loss_risk2, 2))
print(' ')

Сопоставим полученные параметры по регионам (0|1|2):
 
Средний запас фактического сырья по регионам из выборки 25000 скважин:
92.39 | 68.98 | 94.55
 
RMSE предсказаний по регионам:
37.72 | 0.89 | 39.98
 
Суммарная фактическая прибыль с лучших 200 предсказанных скважин по регионам из выборки 25000 скважин:
3136026056.66 | 2415086696.68 | 2465945792.01
 
Средняя суммарная фактическая прибыль с 200 лучших скважин по региону из 1000 подвыборок по 500 скважин:
431538186.58 | 465108803.27 | 426894654.75
 
95%-й доверительный итервал для средней суммарной прибыли с лучших 200 скважин по региону:
-80924626.51 до 941037638.4 | 80299334.72 до 864399761.38 | -114359139.73 до 936751736.17
 
Риск убытков по региону:
0.06 | 0.01 | 0.05
 


**Вывод:** Так как по условию задачи мы исследуем только 500 скважин из каждого региона, то метрики, полученные из данных почти 25000 скважин использовать для сравнения некорректно. Однако выбор упрощается при изучении рисков убытков и сравнении с допустимым в рамках проекта порогом (2.5%): только регион1 проходит этот отсев со значением параметра в 1%. Также регион 2 единственный имеет 95%й доверительный интервал, лежащий полностью в положительном диапазоне. Для разработки предлагаем взять регион1. 